<a href="https://colab.research.google.com/github/JamesBarciz/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/James_Barciz_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
df = df[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <=2000000)]

In [0]:
df['SALE_DATE']

44       01/01/2019
61       01/01/2019
65       01/02/2019
66       01/02/2019
67       01/02/2019
            ...    
23035    04/30/2019
23036    04/30/2019
23037    04/30/2019
23038    04/30/2019
23039    04/30/2019
Name: SALE_DATE, Length: 13471, dtype: object

In [0]:
# Getting the month value
# df['SALE_DATE'][0].split('/')[0]

In [0]:
months = []

for i in df['SALE_DATE']:
  months.append(int(i.split('/')[0]))

In [0]:
df.insert(0, 'month', months)

In [0]:
df['month'].unique()

array([1, 2, 3, 4])

In [0]:
df.head()

,month,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,1,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,1,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
65,1,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,NaN,R4,"505 WEST 19TH STREET, 7D",7D,10011.0,1.0,0.0,1.0,"18,975",1077.0,2013.0,2,R4,2000000,01/02/2019
66,1,1,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,NaN,D4,"345 EAST 54TH ST, 3B",NaN,10022.0,0.0,0.0,0.0,0,0.0,1960.0,2,D4,330000,01/02/2019
67,1,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,NaN,D4,"16 EAST 80TH STREET, 2A",NaN,10075.0,0.0,0.0,0.0,0,0.0,1925.0,2,D4,600000,01/02/2019


In [0]:
# Reasoning later on in assignment
df = df[df['NEIGHBORHOOD'] != 'OTHER']

In [0]:
# Adding this as well to get rid of NA values
df = df.drop(columns='EASE-MENT')

In [0]:
train = df[df['month'] < 4] # Months: Jan, Feb, Mar
test = df[df['month'] == 4] # Months: Apr

In [0]:
train.select_dtypes(exclude='number').describe().T

,count,unique,top,freq
BOROUGH,1639,3,4,620
NEIGHBORHOOD,1639,10,FLUSHING-NORTH,310
BUILDING_CLASS_CATEGORY,1639,29,10 COOPS - ELEVATOR APARTMENTS,547
TAX_CLASS_AT_PRESENT,1639,8,2,1007
BUILDING_CLASS_AT_PRESENT,1639,50,D4,547
ADDRESS,1639,1631,"2180 33RD ROAD, 15A",2
APARTMENT_NUMBER,472,268,3A,13
LAND_SQUARE_FEET,1639,289,0,926
BUILDING_CLASS_AT_TIME_OF_SALE,1639,50,D4,547
SALE_DATE,1639,65,03/20/2019,55


In [0]:
# Look at catagorical features with minimal unique values (to not add so many columns)

In [0]:
# Get a look at value counts for each

print(df['BOROUGH'].value_counts(),
      df['NEIGHBORHOOD'].value_counts(),
      df['TAX_CLASS_AT_PRESENT'].value_counts()
)

1    804
4    765
3    544
Name: BOROUGH, dtype: int64 FLUSHING-NORTH             384
UPPER EAST SIDE (59-79)    254
UPPER EAST SIDE (79-96)    251
BEDFORD STUYVESANT         224
FOREST HILLS               223
UPPER WEST SIDE (59-79)    187
EAST NEW YORK              170
ASTORIA                    158
BOROUGH PARK               150
GRAMERCY                   112
Name: NEIGHBORHOOD, dtype: int64 2     1308
1      556
2C     102
4       78
2A      38
1C      18
1B       9
1A       4
Name: TAX_CLASS_AT_PRESENT, dtype: int64


In [0]:
# The 'OTHER' value in df['NEIGHBORHOOD'] seems to be a value that needs to
# be removed as there are almost 30 times as many as the next value

# Ran code below before making Train/Test
# print(df[df['NEIGHBORHOOD'] == 'OTHER'].shape)
# df = df[df['NEIGHBORHOOD'] != 'OTHER']

In [0]:
# Yields answers in scientific notation without '.round(2)'
train.groupby('NEIGHBORHOOD')['SALE_PRICE'].mean().round(2)

NEIGHBORHOOD
ASTORIA                     839205.97
BEDFORD STUYVESANT          959051.86
BOROUGH PARK               1044024.52
EAST NEW YORK               580278.19
FLUSHING-NORTH              691111.45
FOREST HILLS                556342.91
GRAMERCY                    979655.32
UPPER EAST SIDE (59-79)     923063.17
UPPER EAST SIDE (79-96)     892333.17
UPPER WEST SIDE (59-79)     945523.20
Name: SALE_PRICE, dtype: float64

In [0]:
# Yields answers in scientific notation without '.round(2)' 
train.groupby('TAX_CLASS_AT_PRESENT')['SALE_PRICE'].mean().round(2)

TAX_CLASS_AT_PRESENT
1      938165.28
1A     715712.50
1B     706875.00
1C     790236.00
2      756189.54
2A    1201856.87
2C     749263.92
4      852934.74
Name: SALE_PRICE, dtype: float64

In [0]:
train.groupby('BOROUGH')['SALE_PRICE'].mean().round(2)

BOROUGH
1    926380.27
3    853978.40
4    683493.40
Name: SALE_PRICE, dtype: float64

In [0]:
# Starting One-Hot Encoding Process
target = 'SALE_PRICE'
high_cardinality = ['BUILDING_CLASS_CATEGORY', 'BUILDING_CLASS_AT_PRESENT',
                    'ADDRESS', 'APARTMENT_NUMBER', 'LAND_SQUARE_FEET',
                    'BUILDING_CLASS_AT_TIME_OF_SALE', 'YEAR_BUILT', 'SALE_DATE'
                   ]
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
print(X_train.shape)
X_train.head()

(1639, 12)


,month,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE
67,1,1,UPPER EAST SIDE (79-96),2,1491,62,10075.0,0.0,0.0,0.0,0.0,2
71,1,1,UPPER WEST SIDE (59-79),2,1171,2200,10069.0,1.0,0.0,1.0,827.0,2
87,1,3,BEDFORD STUYVESANT,2,2016,12,11238.0,28.0,1.0,29.0,26310.0,2
88,1,3,BEDFORD STUYVESANT,2,2016,14,11238.0,13.0,1.0,14.0,12381.0,2
104,1,3,EAST NEW YORK,1,3995,24,11207.0,2.0,0.0,2.0,1800.0,1


In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [0]:
print(X_train.shape)
X_train.head()

(1639, 30)


,month,BOROUGH_1,BOROUGH_3,BOROUGH_4,NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_FOREST HILLS,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_1A,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE
67,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1491,62,10075.0,0.0,0.0,0.0,0.0,2
71,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1171,2200,10069.0,1.0,0.0,1.0,827.0,2
87,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2016,12,11238.0,28.0,1.0,29.0,26310.0,2
88,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2016,14,11238.0,13.0,1.0,14.0,12381.0,2
104,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3995,24,11207.0,2.0,0.0,2.0,1800.0,1


In [0]:
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=15) # k=15 arbitrary from lecture
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.fit_transform(X_test)
X_train_selected.shape

TypeError: ignored